In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import imageio
plt.close('all')
import random
import os
import math
from scipy.optimize import curve_fit
from skimage.draw import disk
from tifffile import imsave
from scipy.integrate import quad

In [15]:
def prob(x0,y0,x,y,a,A):
    r=np.sqrt((x-x0)**2+(y-y0)**2)
    if r==0:
        return 1
    else:
        p=A*np.exp(-(r**a))#A/(r**(a))
        #print(p)
        return p
   
    
def makec(p):
    p=int(p*100)
    t=np.random.random_integers(100)
    #print(t,p)
    if p>t:
        return True
    else:
        return False
    
    
def rgba2rgb( rgba, background=(255,255,255) ):
    row, col, ch = rgba.shape

    if ch == 3:
        return rgba

    assert ch == 4, 'RGBA image has 4 channels.'

    rgb = np.zeros( (row, col, 3), dtype='float32' )
    r, g, b, a = rgba[:,:,0], rgba[:,:,1], rgba[:,:,2], rgba[:,:,3]

    a = np.asarray( a, dtype='float32' ) / 255.0

    R, G, B = background

    rgb[:,:,0] = (r * a + (1.0 - a) * R) /255
    rgb[:,:,1] = (g * a + (1.0 - a) * G) /255
    rgb[:,:,2] = (b * a + (1.0 - a) * B) /255

    return np.asarray( rgb, dtype='uint8' )


def circle(x0,y0,r):
    
    x1=np.linspace(x0-r,x0+r,3000,endpoint=True)
    x2=np.linspace(x0-r,x0+r,3000,endpoint=True)
    y1=np.sqrt(r**2-(x1-x0)**2)+y0
    y2=-1*np.sqrt(r**2-(x1-x0)**2)+y0
    x=np.append(x1,x2)
    y=np.append(y1,y2)
        
    return x,y 

def circletest(x0, y0, r0, x1, y1, r1):
    # circle 1: (x0, y0), radius r0
    # circle 2: (x1, y1), radius r1

    d=np.sqrt((x1-x0)**2 + (y1-y0)**2)
    
    #print('d',d,r1,r0)
    
    # non intersecting
    if d > r0 + r1 and r0 != r1:
        #print('1')
        return 0
    # One circle within other
    if d < abs(r0-r1):
        #print('2')
        return 1
    # coincident circles
    if d == 0 and r0 == r1:
        #print('3')
        return 0
    else:
        #print('4')
        return 1

    
def splitprob(r,low,high):
    if r <= low:
        diff = 0
    else:
        diff = ((1/(high-low))**3 * (r-low)**3)*100
    return 100-diff

def distance(circle_a, circle_b):
    return ((circle_a.x - circle_b.x) ** 2 + (circle_a.y - circle_b.y) ** 2) ** 0.5
    
def circletest_array(x0, y0, r0):
    # circle 1: (x0, y0), radius r0
    # circle 2: (x1, y1), radius r1
    c_m=np.zeros((len(x0),len(x0)))
    
    
    for i in range(len(x0)):
         for j in range(len(x0)):
                c_m[i,j]=circletest(x0[i], y0[i], r0[i], x0[j], y0[j], r0[j])
    
    #print('sum',np.sum(c_m))
    return np.sum(c_m)
    
def ini_pic(b, r_mu, r_sig, pix_max,pix_min, t, run, path='.',bg=False):
    my_dpi = 96
    
    if not os.path.isdir(path+'/' + str(run)):
        os.mkdir(path+'/' + str(run))
        os.mkdir(path+'/' + str(run) + '_GT')
        os.mkdir(path+'/' + str(run) + '_GT'+'/TRA')
    
    fig = plt.figure(figsize=(pix_max / my_dpi, pix_max / my_dpi), dpi=my_dpi)
    ax = fig.gca()
    
    circles = []
    
    x=np.array([])
    y=np.array([])
    x0c=np.array([2])
    y0c=np.array([0.5])
    rc=np.array([0])
    split_prob=np.array([0])
    
    x_pos = np.linspace(1 * r_mu, 1 - 1 * r_mu, 1500)
    y_pos = np.linspace(1 * r_mu, (pix_min/pix_max) - 1 * r_mu, int((pix_min/pix_max)*1500))
    
    for _ in range(b):
        x0 = np.random.choice(x_pos)
        y0 = np.random.choice(y_pos)
        r_com = np.random.normal(r_mu, r_sig, 1)
        while r_com < 34/512:
            r_com = np.random.normal(r_mu, r_sig, 1)
        circle = Circle(x0, y0, r_com)
        circles.append(circle)
    
    resolve_all_overlaps(circles)
    
    # Extract updated x, y, r values for plotting
    xu, yu, ru = zip(*[(circle.x if isinstance(circle.x, float) else circle.x[0], 
                   circle.y if isinstance(circle.y, float) else circle.y[0], 
                   circle.radius if isinstance(circle.radius, float) else circle.radius[0]) for circle in circles])
    print('xu',xu)
    x0c = np.append(x0c,xu)
    y0c = np.append(y0c,yu)
    rc = np.append(rc,ru)
    
    split_prob = [100 - splitprob(radius, 0.035, 0.06) for radius in rc]
    
    for circle in circles:
        circ = plt.Circle((circle.x, circle.y), circle.radius, color='red', fill=False)
        ax.add_patch(circ)
        
    plt.gca().set_aspect('equal', adjustable='box')
    plt.axis('off')
    plt.xlim(0, 1)
    plt.ylim(0, (pix_min/pix_max))
    plt.savefig(path+'/'+str(run)+'/'+str(t)+'.png',dpi=my_dpi,transparent=False) 
    plt.close()
    
    r_count = np.unique(rc)
    ide = np.append(-1, np.arange(1, b + 1))
    '''
    if len(r_count) < b:
        print('!!!!!!!!!!!!!!!!!', len(r_count))
        x0c, y0c, rc, ide, split_prob = ini_pic(b, r_i, pix, t, run, bg=False)
    '''
    return x0c, y0c, rc, ide, split_prob

def gt(x,y,r,t,run,ide,split_prob,path='.',split_id=0,savefile=True,shuffle=False):
    
    
    #plt.rcParams['axes.facecolor'] = 'black'
    my_dpi=96
    
    #print('r',r,np.mean(r))


    t_v=np.ones(len(x))*t
    
    if t==0:
        split_id=np.zeros(len(x))
    
    if shuffle:
        ind = np.argsort(x) 
        #print('ind',ind)
        random.shuffle(ind)
        #print('ind',ind)
        x=np.array(x)
        y=np.array(y)
        r=np.array(r)
        #print('x',x)
        #print('y',y)
        x= np.take_along_axis(x, ind, axis=-1)  
        #print('x2',x)
        
        y=np.take_along_axis(y, ind, axis=-1) 
        
        #print('y2',y)
        #print('ind2',ind)
        r=np.take_along_axis(r, ind, axis=-1)
        ide=np.take_along_axis(ide, ind, axis=-1)
        split_id=np.take_along_axis(split_id, ind, axis=-1)
        split_prob=np.take_along_axis(split_prob, ind, axis=-1)
    
    
    
    if savefile:
        if t==0:
            np.savetxt(path+'/'+str(run)+'_GT'+'/TRA/'+'pos_GT.txt', np.c_[x,y,r,ide,split_id,split_prob,t_v],delimiter='\t',header='x'+'\t'+'y'+'\t'+'r'+'\t'+'id'+'\t'+'split_id'+'\t'+'s_prob'+'\t'+'t')
        else:    
            xl,yl,rl,idel,split_l,s_pr_l,t_vl= np.loadtxt(path+'/'+str(run)+'_GT'+'/TRA/'+'pos_GT.txt',skiprows=1, delimiter='\t', usecols=(0,1,2,3,4,5,6), unpack=True)
            
            xl=np.append(xl,x)
            yl=np.append(yl,y)
            rl=np.append(rl,r)
            t_vl=np.append(t_vl,t_v)
            idel=np.append(idel,ide)
            split_l=np.append(split_l,split_id)
            s_pr_l=np.append(s_pr_l,split_prob)
            np.savetxt(path+'/'+str(run)+'_GT'+'/TRA/'+'pos_GT.txt', np.c_[xl,yl,rl,idel,split_l,s_pr_l,t_vl],delimiter='\t',header='x'+'\t'+'y'+'\t'+'r'+'\t'+'id'+'\t'+'split_id'+'\t'+'s_prob'+'\t'+'t')
    plt.close()

def move(x,y,r,mv_mu,mv_std,sp_prob,t,r_split,pix_max,pix_min,run,ide,path='.',reco=True,forget=True,directed=0):
    
    

    
    vec=np.linspace(0,2*np.pi,500)

    n=pix_max
    my_dpi=96
    fig = plt.figure(figsize=(pix_max/my_dpi, pix_max/my_dpi), dpi=my_dpi)
    
    plt.axis('off')
    
    plt.gca().set_aspect('equal', adjustable='box')
    ax = fig.gca()
    plt.xlim(0,1)
    plt.ylim(0,pix_min/pix_max)
    
    rec_flag=False
    xrec_l=[]
    yrec_l=[]
    rrec_l=[]
    
    for m in range(5):
        
        misdetect=np.random.random_integers(99)
        m_lim=90
    
        if misdetect >= m_lim and reco==True:
            
            coord=np.linspace(0,1,num=500)
            xrec=np.random.choice(coord)
            yrec=np.random.choice(coord)
            rrec=np.random.choice(coord)*10
            #rec=plt.Rectangle((xrec,yrec), rrec*10**(-2), rrec*10**(-2),fc=None,ec="blue") 
            #rec=plt.Circle((xrec,yrec), rrec*10**(-2), color='red',fill=False)
            #ax.add_patch(rec)
            
            xrec_l.append(xrec)
            yrec_l.append(yrec)
            rrec_l.append(rrec)
            
            print('rec!',len(xrec_l))
           
            
            rec_flag=True

    
    #xp=np.array([0,1])
    #yp=np.array([0,1])
    
    pl=True
    
    

        

    re = np.array([])
    x0c=np.array([])
    y0c=np.array([])
    r0c=np.array([])
    idec=np.array([])
    split_id=np.array([])
    split_prob=np.array([])
    flag=0

       
    for i in range(len(x)):
        phi=np.random.choice(vec)
        mv = sample_from_power_law(mv_mu, mv_std, size=1)#rejection_sample(mv_mu, mv_sig)
        x_n=x[i]+(mv/pix_max)*np.sin(phi)
        y_n=y[i]+(mv/pix_max)*np.cos(phi)
        
        
        r_e0=0.1 #!!!!!!!!
        if r[i] < 0.35*r_mu:
            r_e0 = np.random.random_integers(8,12)
            
        re=np.append(re,r_e0)
        #print('ri0',r[i])
        r[i] = r[i]*(1+(r_e0/100))
        #print('ri1',r[i])

        if x_n < 0 or x_n > 1:
            x_n=x[i]-(mv/pix_max)*np.sin(phi)

        if y_n < 0 or y_n > pix_min/pix_max:
            y_n=y[i]-(mv/pix_max)*np.cos(phi)


        sp=np.random.random_integers(99)
        #print('ri',r[i])
        if percent_true(sp_prob) and t>7 and r[i]>r_split and t<10:###splitprob(r[i],0.035,0.06): ###96
            #print(r[i],'split',splitprob(r[i],0.035,0.06))
            flag=flag+1
            d=random.uniform(0.7, 0.8)
            x_n1, y_n1, x_n2, y_n2 = split_cell_axis(x_n, y_n, r[i], d)
            
            
            #x_n1=x_n-1.05*r[i]
            #x_n2=x_n+1.05*r[i]


            #r_e=np.random.random_integers(5)

            #r_n1= r[i]*(1+(r_e/100))
            #r_n2= r[i]*(1-(r_e/100))

            #print('r_n1')
            #r_e1=np.random.random_integers(15,40)
            #r_e2=np.random.random_integers(15,40)
            
            r_e1=np.random.random_integers(40,45)
            

            z=0
            while z<100:


                r_n1= r[i]*(1-(r_e1/100))
                r_n2= r[i]*(1-(r_e1/100))
                #print('r_n1',r_n1)
                #print('r_n2',r_n2)
                if r_n1 in r or r_n2 in r:
                    #print('while',r)
                    r_e1=np.random.random_integers(40,65)
                    #print('r_e',r_e)
                    #r_n1= r[i]*(1+(r_z/100))+r_n1/100
                    #r_n2= r[i]*(1-(r_z/100))+r_n2/100


                    #print()
                    z=z+1
                else:

                    break


            circ1=plt.Circle((x_n1,y_n1), r_n1, color='red',fill=False)
            circ2=plt.Circle((x_n2,y_n2), r_n2, color='red',fill=False)
            ax.add_patch(circ1)
            ax.add_patch(circ2)
            x0c=np.append(x0c,x_n1)
            y0c=np.append(y0c,y_n1)
            x0c=np.append(x0c,x_n2)
            y0c=np.append(y0c,y_n2)

            split_id=np.append(split_id,ide[i])
            split_id=np.append(split_id,ide[i])
            
            split_prob=np.append(split_prob,100-splitprob(r_n1,0.035,0.06))
            split_prob=np.append(split_prob,100-splitprob(r_n2,0.035,0.06))


            if len(idec)!=0:
                max_id=np.max(np.append(ide,idec))
            else:
                max_id=np.max(ide)


            idec=np.append(idec,max_id+1)
            idec=np.append(idec,max_id+2)

            r0c=np.append(r0c,r_n1)
            r0c=np.append(r0c,r_n2)
        else:
            #print(r[i],'nosplit',splitprob(r[i],0.035,0.06))
            split_id=np.append(split_id,0)
            circ=plt.Circle((x_n,y_n), r[i], color='red',fill=False)
            ax.add_patch(circ)
            x0c=np.append(x0c,x_n)
            y0c=np.append(y0c,y_n)
            r0c=np.append(r0c,r[i])
            split_prob=np.append(split_prob,100-splitprob(r[i],0.035,0.06))

            idec=np.append(idec,ide[i])

    circles = [Circle(x, y, r) for x, y, r in zip(x0c, y0c, r0c)]

    # Resolve overlaps
    resolve_all_overlaps(circles)

    # Update your circle lists with new values after resolving overlaps
    x0c = [circle.x for circle in circles]
    y0c = [circle.y for circle in circles]
    r0c = [circle.radius for circle in circles]

                
    
    if rec_flag:
        n_miss=len(xrec_l)
        for k in range(len(xrec_l)):
            
            xrec=xrec_l[k]
            yrec=yrec_l[k]
            rrec=rrec_l[k]
            
            x0c=np.append(x0c,xrec)
            y0c=np.append(y0c,yrec)
            r0c=np.append(r0c,rrec*10**(-2))
            #print('rec_split',rrec,100-splitprob(rrec*10**(-2),0.035,0.06))
            split_prob=np.append(split_prob,100-splitprob(rrec*10**(-2),0.035,0.06))
            split_id=np.append(split_id,-1)
            idec=np.append(idec,-2)
    
    
    n_miss=0
        
    #plt.scatter(xp,yp,s=pix**2/n**2,c='blue')
    
    if pl != False:
        plt.savefig(path+'/'+str(run)+'/'+str(t)+'.png',dpi=my_dpi,transparent=False) 
        plt.close()
        
    return x0c, y0c, r0c,idec,split_id,split_prob
        

#plt.ioff()      




    
def sample_from_power_law(n, b=10, size=1):
    A = find_A(n, b)
    u = np.random.rand(size)
    samples = [inverse_cdf(ui, n, A, b) for ui in u]
    return np.array(samples)

def power_law_pdf(x, n, A):
    return A / (x ** n)

# Normalization constant A for defined interval [1, b]
def find_A(n, b=10):
    integral, _ = quad(power_law_pdf, 1, b, args=(n, 1))
    return 1 / integral

# CDF of the power-law function
def power_law_cdf(x, n, A):
    def integrand(t):
        return power_law_pdf(t, n, A)
    result, _ = quad(integrand, 1, x)
    return result

# Inverse of CDF
def inverse_cdf(u, n, A, b=10):
    # Using a simple binary search for the inverse
    low, high = 1, b
    while high - low > 1e-6:
        mid = (low + high) / 2
        if power_law_cdf(mid, n, A) < u:
            low = mid
        else:
            high = mid
    return (low + high) / 2



def split_cell_axis(x_n, y_n, r, d):
    """
    Calculates new positions for the daughter cells of a splitting cell.

    Parameters:
    x_n (float): x position of the parent cell
    y_n (float): y position of the parent cell
    r (float): radius of the parent cell
    theta (float): orientation angle of the split in radians

    Returns:
    tuple: new x and y positions for both daughter cells
    """
    theta = random.uniform(0, 2 * np.pi)
    
    # Displacement along the axis
    delta_x = d * r * np.cos(theta)
    delta_y = d * r * np.sin(theta)

    x_n1 = x_n - delta_x
    y_n1 = y_n - delta_y

    x_n2 = x_n + delta_x
    y_n2 = y_n + delta_y

    return x_n1, y_n1, x_n2, y_n2



def adjust_radius(filepath):
    # Read the data from the file
    with open(filepath, 'r') as file:
        lines = file.readlines()

    # Remove header and store it
    header = lines.pop(0)

    # Process data lines
    data = [list(map(float, line.split())) for line in lines]

    # Find parent cells of all splits
    split_parents = {}
    for line in data:
        split_id = int(line[4])
        if split_id != 0:
            split_parents[split_id] = int(line[6])  # key = split_id, value = split time

    # Adjust radius for each parent cell
    for parent_id, split_time in split_parents.items():
        for line in data:
            cell_id = int(line[3])
            time = int(line[6])

            if cell_id == parent_id:
                if time == split_time - 6:
                    initial_radius = line[2]
                    half= random.uniform(1.6, 2.1)
                    quart = random.uniform(3.6, 4.2)
                if split_time - 6 <= time < split_time:
                    # Adjust the radius
                    
                    if split_time - 6 <= time <= split_time - 5:
                        line[2] = initial_radius / half
                    else:
                        line[2] = initial_radius / quart

    # Convert back to string format and write to file
    with open(filepath, 'w') as file:
        file.write(header)
        for line in data:
            str_line = "\t".join(map(str, line)) + '\n'
            file.write(str_line)

# Test the function


def plot_radius_over_time(filepath, cell_id):
    # Read the data from the file
    with open(filepath, 'r') as file:
        lines = file.readlines()

    # Remove header
    lines.pop(0)

    # Process data lines
    data = [list(map(float, line.split())) for line in lines]

    # Extract time and radius for the specified cell ID
    times = []
    radii = []
    for line in data:
        if int(line[3]) == cell_id:
            times.append(int(line[6]))
            radii.append(line[2])

    # Plot
    plt.plot(times, radii, marker='o', linestyle='-', color='b')
    plt.xlabel('Time')
    plt.ylabel('Radius')
    plt.title(f'Radius Over Time for Cell ID {cell_id}')
    plt.grid(True)
    plt.show()

    
    
    
def resolve_all_overlaps(circles, max_iterations=100000, WIDTH=100, HEIGHT=100):
    for k in range(max_iterations):
        any_overlap = False
        for i, circle_a in enumerate(circles):
            for j, circle_b in enumerate(circles):
                if i != j and circle_a.intersects(circle_b):
                    any_overlap = True
                    overlap_distance = circle_a.radius + circle_b.radius - distance(circle_a, circle_b)
                    move_distance = (overlap_distance * (circle_a.radius + circle_b.radius) / 2) + 0.01

                    # Calculate angle of repulsion
                    angle = math.atan2(circle_a.y - circle_b.y, circle_a.x - circle_b.x)

                    # Add noise to the angle
                    noise = random.uniform(-0.1, 0.1)  # Adjust this value for more or less noise
                    angle += noise

                    # Calculate the forces for the two circles
                    force_a = move_distance + random.uniform(0, 0.2 * move_distance)
                    force_b = move_distance + random.uniform(0, 0.2 * move_distance)

                    # Update circle positions based on the repulsion force
                    circle_a.x += math.cos(angle) * force_a
                    circle_a.y += math.sin(angle) * force_a

                    circle_b.x -= math.cos(angle) * force_b
                    circle_b.y -= math.sin(angle) * force_b

                    # Ensure circle centers stay inside boundaries
                    circle_a.x = max(circle_a.radius, min(WIDTH - circle_a.radius, circle_a.x))
                    circle_a.y = max(circle_a.radius, min(HEIGHT - circle_a.radius, circle_a.y))

                    circle_b.x = max(circle_b.radius, min(WIDTH - circle_b.radius, circle_b.x))
                    circle_b.y = max(circle_b.radius, min(HEIGHT - circle_b.radius, circle_b.y))

        if not any_overlap:
            #print('K',k)
            break

# Test the function
class Circle:
    def __init__(self, x, y, radius):
        self.x = x
        self.y = y
        self.radius = radius

    def __repr__(self):
        return f"Circle({self.x}, {self.y}, {self.radius})"
    
    def intersects(self, other):
        """Check if this circle intersects with another circle."""
        distance = ((self.x - other.x) ** 2 + (self.y - other.y) ** 2) ** 0.5
        return distance < (self.radius + other.radius)

    def bounding_box(self):
        """Return the bounding box of the circle."""
        return Rectangle(self.x - self.radius, self.y - self.radius, self.x + self.radius, self.y + self.radius)
    
    
def rejection_sample(mv_mu, mv_sig):
    while True:
        mv = np.random.normal(mv_mu, mv_sig, 1)
        if mv >= 0:
            return mv
        
def compute_radius_stats(mu_area, sigma_area):
    mu_radius = np.sqrt(mu_area / np.pi)
    sigma_radius = sigma_area / (2 * np.sqrt(np.pi * mu_area))
    return mu_radius, sigma_radius

def percent_true(p):
    return random.random() < p


def func_neg(x, a, b, c):
    return (1 - a * (b ** x)) + c

def lin_func_neg(x,m):
    return 1-m*x

def adjust_radius_using_func(time_since_split, initial_radius, popt_neg):
    size_ratio = func_neg(time_since_split, *popt_neg)
    return initial_radius * size_ratio

def adjust_radius_using_lin_func(time_since_split, initial_radius, m):
    size_ratio = lin_func_neg(time_since_split,m)
    return initial_radius * size_ratio

def modify_file_radius(filepath, popt_neg):
    # Read the data from the file
    with open(filepath, 'r') as file:
        lines = file.readlines()

    # Remove header and store it
    header = lines.pop(0)

    # Process data lines
    data = [list(map(float, line.split())) for line in lines]

    # Find parent cells of all splits
    split_parents = {}
    for line in data:
        split_id = int(line[4])
        if split_id != 0 and split_id != -1:
            split_parents[split_id] = int(line[6])  # key = split_id, value = split time

    # Adjust radius for each parent cell
    for parent_id, split_time in split_parents.items():
        for line in data:
            cell_id = int(line[3])
            time = int(line[6])

            if cell_id == parent_id:
                time_since_split = time - split_time
                if time_since_split == -5:
                    initial_radius = line[2]
                if -5<= time_since_split < 0:
                    # Adjust the radius using the func_neg
                    line[2] = adjust_radius_using_lin_func(time_since_split, initial_radius, popt_neg)#adjust_radius_using_func(time_since_split, initial_radius, popt_neg)

    # Convert back to string format and write to file
    with open(filepath, 'w') as file:
        file.write(header)
        for line in data:
            str_line = "\t".join(map(str, line)) + '\n'
            file.write(str_line)

            
def two_gaussians(x, A1, mu1, sigma1, A2, mu2, sigma2):
    g1 = A1 * np.exp(-(x - mu1)**2 / (2 * sigma1**2))
    g2 = A2 * np.exp(-(x - mu2)**2 / (2 * sigma2**2))
    return g1 + g2

def convert_to_CTC(filename,image_shape):
    
    dir_path = os.path.dirname(filename)
    
    # Load data from the file
    xl, yl, rl, idel, split_l, s_pr_l, t_vl = np.loadtxt(filename, skiprows=2, delimiter='\t', unpack=True)

    # Convert tracking format
    tracks = {}
    for x, y, r, id_, split_id, t in zip(xl, yl, rl, idel, split_l, t_vl):
        if id_ not in tracks:
            tracks[id_] = {'start': int(t), 'end': int(t), 'parent': int(split_id)}
        else:
            tracks[id_]['end'] = int(t)
    
    # Save the tracking format
    with open(os.path.join(dir_path, "man_track.txt"), "w") as out_file:
        for id_, details in tracks.items():
            out_line = f"{int(id_)} {details['start']} {details['end']} {details['parent']}\n"
            out_file.write(out_line)

    # Create a dictionary to track cells at each time point
    time_dict = {}
    for x, y, r, id_, t in zip(xl, yl, rl, idel, t_vl):
        if t not in time_dict:
            time_dict[t] = []
        time_dict[t].append((x, y, r, id_))

    width, height = image_shape

    for t, cells in time_dict.items():
        # Create a blank 16-bit image
        img = np.zeros((height, width), dtype=np.uint16)

        for x, y, r, id_ in cells:
            y_img = int(x * height)
            x_img = int((1 - y) * width)
            r=12 #!!!
            rr, cc = disk((x_img, y_img), r)
        
            # Clip the coordinates to ensure they are within the image boundaries
            rr = np.clip(rr, 0, height-1)
            cc = np.clip(cc, 0, width-1)

            img[rr, cc] = int(id_)
        # Save the image as a 16-bit TIFF
        imsave(os.path.join(dir_path, f"man_track{int(t)}.tif"), img)


In [13]:
a=np.array([4,4,5,3,1])
b=np.array([1,2,3,4,5])
print(a[1:])

[4 5 3 1]


In [29]:
#print(stop)
#forget=True
pix_max=512
pix_min=512


non_dep_area=np.array([10001.570812071384,1500.570812071384])



#mv_mu=0.523662
#mv_sig=0.508994
n_powerlaw=0.77369779038092
max_d=15

sp_mu=0.834381/100 #3.
sp_sig=0.552273/100
path0='./HeLa_art_2'
combined_radii=[]
for i in range(50,100):
    
    
    max_area=10000
    r_split=np.sqrt(max_area/np.pi)/pix_max
    A_mu=non_dep_area[0]
    A_sig=non_dep_area[1]
    sp_prob=np.random.normal(sp_mu, sp_sig, 1)
    sp_prob=0#0.15 #!!!!!!!!!!!
    print('iiiiiiiiiiiiii',i,sp_prob)
    t=30 #30
    b=random.randint(7, 11)
    r_mu,r_sig= compute_radius_stats(A_mu, A_sig)
    print('r_mu',r_mu,r_sig)
    r_mu=r_mu/pix_max
    r_sig=r_sig/pix_max

    run=i
    x_f, y_f, r_f,idec_f,split_prob_f=ini_pic(b,r_mu,r_sig,pix_max,pix_min,0,run,path=path0)
   
    #print(len(x), len(y), len(r), len(ide), len(split_prob))
    gt(x_f,y_f,r_f,0,run,idec_f,split_prob_f,path=path0)
    #mv=0.02
    

    x_f=x_f[idec_f>0]
    y_f=y_f[idec_f>0]
    r_f=r_f[idec_f>0]
    idec_f=idec_f[idec_f>0]

    
    split_id_f = np.zeros(len(x_f))
    # Convert to numpy array for boolean indexing

    for l in range(1, t):

        mask = split_id_f != -1
        x_f = np.array(x_f)[mask]
        y_f = np.array(y_f)[mask]
        r_f = np.array(r_f)[mask]
        idec_f = np.array(idec_f)[mask]
        split_id_f = np.array(split_id_f)[mask]


        
        x_f, y_f, r_f,idec_f,split_id_f,split_prob_f = move(x_f,y_f,r_f,n_powerlaw,max_d,sp_prob,l,r_split,pix_max,pix_min,run,idec_f,forget=False,directed=0,path=path0)

            
        #print(l,len(x),ide)
        #x,y,r = ini_pic(b,r_i,pix,l,run)
        gt(x_f,y_f,r_f,l,run,idec_f,split_prob_f,split_id=split_id_f,path=path0,shuffle=True)
        
    filepath=path0+'/'+str(i)+'_GT'+'/TRA/'+'pos_GT.txt'
    #popt_neg=[0.55618812, 1.54775129, 0.01123954]
    popt_neg=0.05
    modify_file_radius(filepath, popt_neg)
    convert_to_CTC(filepath,(512,512))
    data = np.loadtxt(filepath, delimiter='\t', skiprows=1, usecols=(2))
    
    combined_radii.extend(data)
    

iiiiiiiiiiiiii 50 0
r_mu 56.423389359817484 4.232699681998557
xu (0.5861396176163499, 0.7584954583751555, 0.309253544684902, 0.5429063917033623, 0.8907588322779679, 0.2849479640079963, 0.1859281683585172, 0.7270162473216901, 0.5614045778743967, 0.4589779087231209, 0.33521031148128166)
rec! 1


/tmp/ipykernel_58127/177910083.py:244: DeprecationWarning: This function is deprecated. Please call randint(1, 99 + 1) instead
  misdetect=np.random.random_integers(99)
/tmp/ipykernel_58127/177910083.py:309: DeprecationWarning: This function is deprecated. Please call randint(1, 99 + 1) instead
  sp=np.random.random_integers(99)


rec! 1
rec! 1
rec! 1
rec! 1
rec! 2
rec! 1
rec! 1
rec! 2
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 2
iiiiiiiiiiiiii 51 0
r_mu 56.423389359817484 4.232699681998557
xu (0.5531256248500717, 0.8941108130977137, 0.15232820783396742, 0.6429996993902802, 0.7686266747974426, 0.32135338727145524, 0.48662871314111134, 0.37487915688721724, 0.5490311530527529, 0.6958413304598429, 0.17021452052697092)
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 2
rec! 1
rec! 1
rec! 1
rec! 1
rec! 2
rec! 3
iiiiiiiiiiiiii 52 0
r_mu 56.423389359817484 4.232699681998557
xu (0.43941097414010055, 0.9321435555840393, 0.5231199722608468, 0.21814956026628163, 0.8180273760800731, 0.25958625783768036, 0.5012103723012157, 0.7395441259437592, 0.2931618964363421)
rec! 1
rec! 2
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
iiiiiiiiiiiiii 53 0
r_mu 56.423389359817484 4.232699681998557
xu (0.4591739181974068, 0.48764426510075654, 0.33990324002066336, 0.7808003950644222, 0.16281021953792865, 0.248693605

rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
iiiiiiiiiiiiii 75 0
r_mu 56.423389359817484 4.232699681998557
xu (0.2590600553096102, 0.37128082488991315, 0.1464722699226177, 0.17313701692002537, 0.6937288595091206, 0.8320694679102645, 0.07120205421341344)
rec! 1
rec! 1
rec! 2
rec! 1
rec! 1
rec! 1
rec! 1
iiiiiiiiiiiiii 76 0
r_mu 56.423389359817484 4.232699681998557
xu (0.2799238016785905, 0.7685993543442615, 0.3530044579371018, 0.12671798280951035, 0.5378036431357952, 0.9036663490520174, 0.5843228325670129, 0.7005344772784313, 0.13130574025614195, 0.49419562309102655, 0.946109788180055)
rec! 1
rec! 2
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 1
rec! 2
rec! 1
rec! 1
iiiiiiiiiiiiii 77 0
r_mu 56.423389359817484 4.232699681998557
xu (0.6404143243836181, 0.9638931134539269, 0.1942545110018496, 0.6006538758520081, 0.7591911896043506, 0.861208080252603, 0.24775331628984426, 0.3991000477963974, 0.3379958664776079)
rec! 1
rec! 1
rec! 1
rec! 1
rec! 2
rec! 1
rec! 2
rec! 1
rec! 2
rec! 

OSError: [Errno 28] No space left on device

In [ ]:
combined_radii=np.array(combined_radii)

combined_radii = np.pi*(combined_radii*512)**2 #area

counts, bins, _ = plt.hist(combined_radii, bins=100, edgecolor='k')
bin_centers = (bins[:-1] + bins[1:]) / 2

# 2. Fit the function to the histogram data
popt, pcov = curve_fit(two_gaussians, bin_centers, counts, maxfev=5000,p0=(1200, 400, 50, 400, 750, 150))
print("Fitted parameters:", popt)

# 3. Plot the histogram and overlay the fitted function
plt.hist(combined_radii, bins=100, edgecolor='k', label="Data", alpha=0.6)
plt.plot(bin_centers, two_gaussians(bin_centers, *popt), color="red", label="Fitted Function")

plt.xlabel('Area')
plt.ylabel('Frequency')
plt.title('Histogram of Radii with Fitted Function')
plt.legend()
plt.show()
# radius =  map_value_linear(value, 0.003, 0.45, 2, 8)

In [ ]:
filepath = './'+str(1)+'_GT'+'/'+'pos_GT.txt'
cell_id = 7  # replace with your desired cell ID
plot_radius_over_time(filepath, cell_id)


In [ ]:
x=0.034
c=0.03366667
b=[0.03366667,0.034,0.03466667]

print(x in b or c in b)

print(np.arange(1,4))

In [ ]:
run=3
xl,yl,rl= np.loadtxt('./'+str(run)+'_GT'+'/'+'pos_GT.txt',skiprows=1, delimiter='\t', usecols=(0,1,2), unpack=True)

x1=[]
y1=[]
x2_t=[]
y2_t=[]
x2=[]
y2=[]
x3=[]
y3=[]

for i in range(len(xl)):
    if i % 3 == 0:
        x1.append(xl[i])
        y1.append(yl[i])
        
    else:
        x2_t.append(xl[i])
        y2_t.append(yl[i])
        
for i in range(len(x2_t)):
    
    if i % 2 == 0:
        x2.append(x2_t[i])
        y2.append(y2_t[i])
        
    else:
        x3.append(x2_t[i])
        y3.append(y2_t[i])
        
        



plt.plot(x1,y1,c='red')
plt.plot(x2,y2,c='blue')
plt.plot(x3,y3,c='green')
plt.show()